In [1]:
import pandas as pd
from pmdarima import auto_arima
import matplotlib.pyplot as plt

In [2]:
data=pd.read_csv("가구수비율.csv")

In [3]:
# 결과를 저장할 데이터프레임 초기화
forecast_results = []

# 자치구 리스트 추출
districts = data['자치구'].unique()

# 변수 리스트 (예측하고자 하는 변수들)
variables = [
    "공동주택/가구수", "나머지주택/가구수"]

# 각 자치구별로 모델 생성 및 예측
for district in districts:
    district_data = data[data['자치구'] == district]
    
    for variable in variables:
        if variable not in district_data.columns:
            continue

        # 연도별 데이터 정리
        ts_data = district_data[['년도', variable]].set_index('년도')
        ts_data = ts_data.dropna()  # 결측값 제거

        # Auto ARIMA 모델 학습
        model = auto_arima(ts_data, seasonal=True, m=1, suppress_warnings=True, start_p=0, d=1, start_q=0, max_p=3, max_q=3, 
                            enforce_stationarity=False)

        # 예측값이 음수가 나오지 않도록 처리
        def ensure_positive_forecast(forecast):
            return [max(0, value) for value in forecast]

        
        # 예측 (2025, 2026년)
        future_years = [2025, 2026]
        forecast = ensure_positive_forecast(model.predict(n_periods=len(future_years)))

        # 결과 저장
        for year, value in zip(future_years, forecast):
            forecast_results.append({
                '자치구': district,
                '년도': year,
                '변수': variable,
                '예측값': value
            })

# 결과를 데이터프레임으로 변환
forecast_df = pd.DataFrame(forecast_results)

C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\dyj02\anaconda3\Lib\site-packages\statsmodels\tsa\base\tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_predicti

In [4]:
forecast_df

,자치구,년도,변수,예측값
0,강남구,2025,공동주택/가구수,0.629118
1,강남구,2026,공동주택/가구수,0.629118
2,강남구,2025,나머지주택/가구수,0.193087
3,강남구,2026,나머지주택/가구수,0.193087
4,강동구,2025,공동주택/가구수,0.500078
...,...,...,...,...
95,중구,2026,나머지주택/가구수,0.225913
96,중랑구,2025,공동주택/가구수,0.355451
97,중랑구,2026,공동주택/가구수,0.355451
98,중랑구,2025,나머지주택/가구수,0.311401


In [5]:
forecast_df.to_csv('가구수비율 예측.csv',index=False)